# Functional Connectivity
1. [x] Create a BIDS object
2. [x] Read the Atlas
3. [x] Create BIDSDataGrabber Node
4. [x] Create a FC Node which does the following for each subject - 
    5. [x] Extract ROI time series.
    6. [x] Create FC Maps [ROI x Voxels]
7. [x] Create a Join Node to save the paths of the FC files    
   

In [33]:
from bids.grabbids import BIDSLayout
from nipype.interfaces.fsl import (BET, ExtractROI, FAST, FLIRT, ImageMaths,
                                   MCFLIRT, SliceTimer, Threshold,Info)
from nipype.interfaces.afni import Resample
from nipype.interfaces.io import DataSink
from nipype.pipeline import Node, MapNode, Workflow, JoinNode
from nipype.interfaces.utility import IdentityInterface, Function
import os
from os.path import join as opj
from nipype.interfaces import afni
import nibabel as nib
import numpy as np

In [34]:
# Set paths

path_cwd = os.getcwd()
path_split_list = path_cwd.split('/')
s = path_split_list[0:-1]
s = opj('/',*s) # *s converts list to path, # also it is very important to add '/' in the begining so it is read as directory later
base_directory = opj(s,'result')
data_directory = opj(s,"data/ABIDE-BIDS-Preprocessed/NYU/")

mask_file = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')

In [35]:
layout = BIDSLayout(data_directory)

In [36]:
# code to find how many voxels are in the brain region using the mask

num_brain_voxels = 0
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

x_dim, y_dim, z_dim = mask_data.shape

for i in range(x_dim):
    for j in range(y_dim):
        for k in range(z_dim):
            if mask[i,j,k] == 1:
                num_brain_voxels = num_brain_voxels + 1


## Atlas

In [37]:
# Put in the path of atlas you wish to use
atlasPath = opj(s,'atlas/Full_brain_atlas_thr0-2mm/fullbrain_atlas_thr0-2mm.nii.gz')

In [38]:
# Read the atlas
atlasObject = nib.load(atlasPath)
atlas = atlasObject.get_data()

### Lets see the number of regions in the atlas and display the atlas.

In [39]:
num_ROIs = int((np.max(atlas) - np.min(atlas) ))

print('Min Index:', np.min(atlas),'Max Index', np.max(atlas))
print('Total Number of Parcellations = ',num_ROIs)

Min Index: 0.0 Max Index 274.0
Total Number of Parcellations =  274


In [40]:
number_of_subjects = len(layout.get_subjects()) # Number of subjects you wish to preprocess

## Read the data files using BIDSGrabber

In [41]:
# List of Subject IDs

subject_list = (layout.get_subjects())[0:number_of_subjects]

Create our own custom function - BIDSDataGrabber using a Function Interface.

In [42]:
def get_nifti_filenames(subject_id,data_dir):
#     Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    anat_file_path = [f.filename for f in layout.get(subject=subject_id, type='T1w', extensions=['nii', 'nii.gz'])]
    func_file_path = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    
    return anat_file_path[0],func_file_path[0]

# Refer to Supplementary material section One for info on arguments for layout.get()

Wrap it inside a Node

In [43]:
BIDSDataGrabber = Node(Function(function=get_nifti_filenames, input_names=['subject_id','data_dir'],
                                output_names=['anat_file_path','func_file_path']), name='BIDSDataGrabber')
BIDSDataGrabber.iterables = [('subject_id',subject_list)]
BIDSDataGrabber.inputs.data_dir = data_directory

In [44]:
# To test the function wrapped in the node

# BIDSDataGrabber.inputs.data_dir = data_directory
# BIDSDataGrabber.inputs.subject_id = layout.get_subjects()[0] # gives the first subject's ID
# res = BIDSDataGrabber.run()

# res.outputs

## Create a FC node

This node:
1. Exracts the average time series of the brain ROI's using the atlas and stores 
    it as a matrix of size [ROIs x Volumes].
2. Extracts the Voxel time series and stores it in matrix of size [Voxels x Volumes]


In [45]:
def pear_coff(in_file, atlas_file, mask_file, num_brain_voxels):
    # code to find how many voxels are in the brain region using the mask
    
        # imports
    import numpy as np
    import nibabel as nib
    import os
    from os.path import join as opj

    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()

    x_dim, y_dim, z_dim = mask_data.shape

                    
    atlasPath = atlas_file
    # Read the atlas
    atlasObject = nib.load(atlasPath)
    atlas = atlasObject.get_data()
    
    num_ROIs = int((np.max(atlas) - np.min(atlas) ))


    # Read the brain in_file

    brain_data = nib.load(in_file)
    brain = brain_data.get_data()

    x_dim, y_dim, z_dim, num_volumes = brain.shape
    
    # Initialize a matrix of ROI time series and voxel time series

    ROI_matrix = np.zeros((num_ROIs, num_volumes))
    voxel_matrix = np.zeros((num_brain_voxels, num_volumes))
    
    # Fill up the voxel_matrix 

    voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    voxel_matrix[voxel_counter,:] = brain[i,j,k,:] 
                    voxel_counter = voxel_counter + 1

                    
    # Fill up the ROI_matrix
    # Keep track of number of voxels per ROI as well by using an array - num_voxels_in_ROI[]

    num_voxels_in_ROI = np.zeros((num_ROIs,1)) # A column arrray containing number of voxels in each ROI

    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                label = int(atlas[i,j,k]) - 1
                if label != -1:
                    ROI_matrix[label,:] = np.add(ROI_matrix[label,:], brain[i,j,k,:])
                    num_voxels_in_ROI[label,0] = num_voxels_in_ROI[label,0] + 1

    ROI_matrix = np.divide(ROI_matrix,num_voxels_in_ROI) # Check if divide is working correctly

    X, Y = ROI_matrix, voxel_matrix
    # Subtract mean from X and Y

    X = np.subtract(X, np.mean(X))
    Y = np.subtract(Y, np.mean(Y))

    temp1 = np.dot(X,Y.T)
    temp2 = np.sqrt(np.multiply(X,X))
    temp3 = np.sqrt(np.multiply(Y,Y)).T
    temp4 = np.dot(temp2,temp3)
    coff_matrix = np.divide(temp1, (temp4+0.0001))
    print ("Pear Done")
    sub_id = in_file.split('/')[-1].split('.')[0].split('_')[0].split('-')[1]
    
    fc_file_name = sub_id + '_fc_map'
    
    np.save(fc_file_name, coff_matrix)
    
    
    fc_file_name = fc_file_name + '.npy'
    
    coff_matrix_file = opj(os.getcwd(),fc_file_name) # path
    return coff_matrix_file



In [46]:
# Again Create the Node and set default values to paths

pearcoff = Node(Function(function=pear_coff, input_names=['in_file','atlas_file','mask_file','num_brain_voxels'],
                                output_names=['coff_matrix_file']), name='pearcoff')

pearcoff.inputs.atlas_file = atlasPath
pearcoff.inputs.num_brain_voxels = num_brain_voxels
pearcoff.inputs.mask_file = mask_file

In [47]:
# %%time
# pearcoff.run()

## Datasink
I needed to define the structure of what files are saved and where.

In [48]:
# Create DataSink object
dataSink = Node(DataSink(), name='datasink')

# Name of the output folder
dataSink.inputs.base_directory = opj(base_directory,'datasink_fc')




To create the substitutions I looked the `datasink` folder where I was redirecting the output. I manually selected the part of file/folder name that I wanted to change and copied below to be substituted.  


In [49]:
# Define substitution strings so that the data is similar to BIDS
substitutions = [('_subject_id_', 'sub-')]

# Feed the substitution strings to the DataSink node
dataSink.inputs.substitutions = substitutions

# ('_resample_brain_flirt.nii_brain', ''),
# ('_roi_st_mcf_flirt.nii_brain_flirt', ''),
                

### Following is a Join Node that collects the FC file paths and saves them in a file

In [50]:
def save_fc_file_list_func(fc_file_list):
    # Imports
    import numpy as np
    import os
    from os.path import join as opj
    
    
    print('######################## File List ######################: \n',fc_file_list)
    fc_file_list = np.asarray(fc_file_list)
    np.save('fc_file_list',fc_file_list)
    fc_file_name = 'fc_file_list.npy'
    fc_file_path = opj(os.getcwd(),fc_file_name) # path
    return fc_file_path
    

In [51]:
save_fc_file_list = JoinNode(Function(function=save_fc_file_list_func, input_names=['fc_file_list'],
                 output_names=['fc_file_path']),
                 joinsource="BIDSDataGrabber",
                 joinfield="fc_file_list",
                 name="save_fc_file_list")

### Create the Workflow

In [52]:
wf = Workflow(name="functionalConnectivity")
base_dir = opj(s,'result')
wf.base_dir = base_dir # Dir where all the outputs will be stored(inside BETFlow folder).

wf.connect([      (BIDSDataGrabber, pearcoff, [('func_file_path','in_file')]),
                  (pearcoff,dataSink,[('coff_matrix_file','fc_maps.@coff_matrix_file')]),
                  (pearcoff,save_fc_file_list,[('coff_matrix_file','fc_file_list')]),
                  (save_fc_file_list,dataSink,[('fc_file_path','fc_maps_paths@fc_file_path')])
           ])
# Run it in parallel
%time wf.run('MultiProc', plugin_args={'n_procs': 8})

171031-15:17:27,670 workflow INFO:
	 Workflow functionalConnectivity settings: ['check', 'execution', 'logging']
171031-15:17:28,316 workflow INFO:
	 Running in parallel.
171031-15:17:28,399 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a183 ID: 0
171031-15:17:28,401 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a183 jobid: 0
171031-15:17:28,403 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a182 ID: 2
171031-15:17:28,406 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a182 jobid: 2
171031-15:17:28,409 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a181 ID: 4
171031-15:17:28,411 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a181 jobid: 4
171031-15:17:28,413 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a180 ID: 6
171031-15:17:28,416 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a180 jobid: 6
171031-15:17:28,418 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a179 ID: 8
171031-15:17:28,420 workflow INFO:
	 [Job finished] jobname: BI

171031-15:17:28,701 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a158 ID: 50
171031-15:17:28,703 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a158 jobid: 50
171031-15:17:28,705 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a157 ID: 52
171031-15:17:28,707 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a157 jobid: 52
171031-15:17:28,710 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a156 ID: 54
171031-15:17:28,712 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a156 jobid: 54
171031-15:17:28,713 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a155 ID: 56
171031-15:17:28,715 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a155 jobid: 56
171031-15:17:28,717 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a154 ID: 58
171031-15:17:28,718 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a154 jobid: 58
171031-15:17:28,720 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a153 ID: 60
171031-15:17:28,722 workflow INFO:
	 [Job finished]

171031-15:17:28,904 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a132 ID: 102
171031-15:17:28,920 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a132 jobid: 102
171031-15:17:28,922 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a131 ID: 104
171031-15:17:28,924 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a131 jobid: 104
171031-15:17:28,926 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a130 ID: 106
171031-15:17:28,928 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a130 jobid: 106
171031-15:17:28,929 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a129 ID: 108
171031-15:17:28,930 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a129 jobid: 108
171031-15:17:28,932 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a128 ID: 110
171031-15:17:28,934 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a128 jobid: 110
171031-15:17:28,939 workflow INFO:
	 Executing: pearcoff.a135 ID: 97
171031-15:17:28,941 workflow INFO:
	 [Job finished]

171031-15:17:29,116 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a106 ID: 154
171031-15:17:29,117 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a106 jobid: 154
171031-15:17:29,119 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a105 ID: 156
171031-15:17:29,125 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a105 jobid: 156
171031-15:17:29,127 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a104 ID: 158
171031-15:17:29,128 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a104 jobid: 158
171031-15:17:29,134 workflow INFO:
	 Executing: pearcoff.a111 ID: 145
171031-15:17:29,136 workflow INFO:
	 [Job finished] jobname: pearcoff.a111 jobid: 145
171031-15:17:29,138 workflow INFO:
	 Executing: pearcoff.a110 ID: 147
171031-15:17:29,140 workflow INFO:
	 [Job finished] jobname: pearcoff.a110 jobid: 147
171031-15:17:29,141 workflow INFO:
	 Executing: pearcoff.a109 ID: 149
171031-15:17:29,144 workflow INFO:
	 [Job finished] jobname: pearcoff.a109 jobid: 149
1710

171031-15:17:29,350 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a081 jobid: 204
171031-15:17:29,352 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a080 ID: 206
171031-15:17:29,353 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a080 jobid: 206
171031-15:17:29,359 workflow INFO:
	 Executing: pearcoff.a087 ID: 193
171031-15:17:29,361 workflow INFO:
	 [Job finished] jobname: pearcoff.a087 jobid: 193
171031-15:17:29,363 workflow INFO:
	 Executing: pearcoff.a086 ID: 195
171031-15:17:29,365 workflow INFO:
	 [Job finished] jobname: pearcoff.a086 jobid: 195
171031-15:17:29,366 workflow INFO:
	 Executing: pearcoff.a085 ID: 197
171031-15:17:29,368 workflow INFO:
	 [Job finished] jobname: pearcoff.a085 jobid: 197
171031-15:17:29,370 workflow INFO:
	 Executing: pearcoff.a084 ID: 199
171031-15:17:29,378 workflow INFO:
	 [Job finished] jobname: pearcoff.a084 jobid: 199
171031-15:17:29,380 workflow INFO:
	 Executing: pearcoff.a083 ID: 201
171031-15:17:29,382 workflow INFO:

...

	 Executing node datasink.a108 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051067/datasink
171031-15:29:24,949 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051067/0051067_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051067/0051067_fc_map.npy
171031-15:29:27,523 workflow INFO:
	 [Job finished] jobname: datasink.a101 jobid: 470
171031-15:29:27,529 workflow INFO:
	 Executing: datasink.a109 ID: 478
171031-15:29:27,653 workflow INFO:
	 Executing node datasink.a109 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051068/datasink
171031-15:29:27,666 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051068/0051068_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051068/0051068_fc_map.npy
171031-15:29:35,720 workflow INFO:
	 [Job finished] jobname: datasink.a103 jobid: 472
171031-15:29:35,731 workflow INFO:
	 Executing: datasink.a110 ID: 479
171031-15:29:37,272 workflow INFO

	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051082/0051082_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051082/0051082_fc_map.npy
171031-15:31:24,73 workflow INFO:
	 [Job finished] jobname: datasink.a119 jobid: 488
171031-15:31:24,83 workflow INFO:
	 Executing: datasink.a125 ID: 494
171031-15:31:25,361 workflow INFO:
	 [Job finished] jobname: datasink.a118 jobid: 487
171031-15:31:25,362 workflow INFO:
	 Executing node datasink.a125 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051084/datasink
171031-15:31:25,378 workflow INFO:
	 Executing: datasink.a126 ID: 495
171031-15:31:25,379 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051084/0051084_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051084/0051084_fc_map.npy
171031-15:31:26,568 workflow INFO:
	 Executing node datasink.a126 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051085/datasink
171031-15:31:26,584 interface INFO:


171031-15:33:28,762 workflow INFO:
	 [Job finished] jobname: datasink.a134 jobid: 503
171031-15:33:28,765 workflow INFO:
	 [Job finished] jobname: datasink.a136 jobid: 505
171031-15:33:28,770 workflow INFO:
	 Executing: datasink.a141 ID: 510
171031-15:33:30,8 workflow INFO:
	 Executing: datasink.a142 ID: 511
171031-15:33:30,10 workflow INFO:
	 Executing node datasink.a141 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051101/datasink
171031-15:33:30,18 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051101/0051101_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051101/0051101_fc_map.npy
171031-15:33:30,416 workflow INFO:
	 Executing: datasink.a143 ID: 512
171031-15:33:30,416 workflow INFO:
	 Executing node datasink.a142 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051102/datasink
171031-15:33:30,454 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051102/0051102_fc_map.npy -> /hom

	 Executing node datasink.a157 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051117/datasink
171031-15:35:11,273 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051117/0051117_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051117/0051117_fc_map.npy
171031-15:35:11,991 workflow INFO:
	 [Job finished] jobname: datasink.a149 jobid: 518
171031-15:35:11,998 workflow INFO:
	 Executing: datasink.a158 ID: 527
171031-15:35:13,119 workflow INFO:
	 Executing node datasink.a158 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051118/datasink
171031-15:35:13,126 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051118/0051118_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051118/0051118_fc_map.npy
171031-15:35:13,132 workflow INFO:
	 [Job finished] jobname: datasink.a151 jobid: 520
171031-15:35:13,137 workflow INFO:
	 Executing: datasink.a159 ID: 528
171031-15:35:13,146 workflow INFO

	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051147/0051147_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051147/0051147_fc_map.npy
171031-15:37:08,255 workflow INFO:
	 [Job finished] jobname: datasink.a163 jobid: 532
171031-15:37:08,260 workflow INFO:
	 Executing: datasink.a174 ID: 543
171031-15:37:09,527 workflow INFO:
	 Executing node datasink.a174 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051148/datasink
171031-15:37:09,670 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051148/0051148_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051148/0051148_fc_map.npy
171031-15:37:32,614 workflow INFO:
	 [Job finished] jobname: datasink.a168 jobid: 537
171031-15:37:32,625 workflow INFO:
	 Executing: datasink.a175 ID: 544
171031-15:37:33,469 workflow INFO:
	 [Job finished] jobname: datasink.a167 jobid: 536
171031-15:37:33,472 workflow INFO:
	 Executing node datasink.a175 in dir: /home1/varunk/result/